In [189]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym import spaces
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
from operator import add
import random
import copy
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import gym
import random
import torch.nn as nn
import torch.nn.functional as F

import tensorflow as tf
from IPython import embed
from gym import wrappers


In [190]:
env = gym.make('MountainCar-v0')
env.reset()
action_space=[0,1]
num_states = env.observation_space.shape[0]
done = False

seed = 6
env.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# while not done:
# #     env.render()

#     observation, reward, done, info = env.step(env.action_space.sample())
#     if done:
#         reward = 0
#     print(reward)

In [191]:
class act_Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(act_Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
#         self.fc2 = nn.Linear(24,12)
        self.fc3 = nn.Linear(64,output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
        
        
        

In [192]:
class critic_Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(critic_Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
#         self.fc2 = nn.Linear(24,12)
        self.fc3 = nn.Linear(64,output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
        
        

In [193]:
env.observation_space.shape[0]
# env.action_space.n

2

In [194]:
class actor_class():
    def __init__(self,action_state, output_state):     
        self.network = act_Net(action_state, output_state)
        self.loss = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.network.parameters(), lr=0.001)
    

In [195]:
class critic_class():
    def __init__(self,action_state, output_state): 
        self.network = critic_Net(action_state, output_state)
        self.loss = nn.MSELoss()
        self.optimizer = optim.Adam(self.network.parameters(), lr=0.001)
    

In [178]:
metrics = {}


In [186]:
actor = actor_class(env.observation_space.shape[0], env.action_space.n)
critic = critic_class(env.observation_space.shape[0], 1)
class A2C():
    def __init__(self, num_episodes = 1000):
        
        self.number_of_episodes = num_episodes
        self.hyper_parameters = {}
        self.hyper_parameters['learning_rate'] = 0.001
        self.hyper_parameters['max_timesteps'] = 100
        self.hyper_parameters['epsilon'] = 1
        self.hyper_parameters['epsilon_decay'] = 1.01010101
        self.hyper_parameters['discount_factor'] = 0.99
        self.hyper_parameters['replay_length'] = 2000
        self.hyper_parameters['batch_size'] = 32
        
        
    def encode_one_hot(self, action, n_actions):
        encoded = np.zeros(n_actions, np.float32)
        encoded[action] = 1
        return encoded

    def train(self):       
        metrics['total_rewards'] = []
        for episode in range(self.number_of_episodes):
            total_training_rewards = 0
            curr_state = env.reset()
            curr_state = curr_state.reshape([1, curr_state.shape[0]])
            done = False
            while not done:

                act_probabilities = actor.network(torch.Tensor(curr_state))#.detach().numpy()[0]
                dist = torch.distributions.Categorical(probs=act_probabilities)
                action = dist.sample()
#                 action = np.random.choice(env.action_space.n, 1, p=act_probabilities)[0]
                encoded_action = self.encode_one_hot(action, env.action_space.n)

                next_state, reward, done, info = env.step(action.detach().numpy()[0])
                next_state = next_state.reshape([1, next_state.shape[0]])

                value_curr = critic.network(torch.Tensor(curr_state))
                value_next = critic.network(torch.Tensor(next_state))


                TD_target = reward + (1 - done) * self.hyper_parameters['discount_factor'] * value_next
                advantage = TD_target - value_curr

                critic_loss = advantage.pow(2).mean()
                critic.optimizer.zero_grad()
                critic_loss.backward()
                critic.optimizer.step()
                
#                 target = act_probabilities[action]*advantage
                actor_loss = -dist.log_prob(action)*advantage.detach()
#                 actor_loss = actor.loss(torch.Tensor([-act_probabilities[action]]), torch.Tensor([-target]))
                actor.optimizer.zero_grad()
                actor_loss.backward()
                actor.optimizer.step()
                                                  
                curr_state = next_state
                total_training_rewards += 1

                if done:
                    metrics['total_rewards'].append(total_training_rewards)
                    if episode%100 == 99:       
                        print(episode, metrics['total_rewards'][-10:])
            if len(metrics['total_rewards'])>200:
                count = 0
                for tr in metrics['total_rewards'][-60:]:
                    if tr>470:
                        count += 1
                if count>50:
                    print(count)
                    break
                        
                        
#                     actor.save_weights(actor_checkpoint_path)
#                     critic.save_weights(critic_checkpoint_path)
    def execute(self,num_episodes):
#         env = gym.make('CartPole-v1')
    #     env = wrappers.Monitor(env, "./gym-results", force=True)
        metrics['rewards_evaluation'] = []
        for episodes in range(num_episodes):
            step_count = 0
            state = env.reset()
            done = False
            while not done:
                act_probabilities = actor.network(torch.Tensor(state)).detach().numpy()
                action = np.argmax(act_probabilities)
#                 print(action, act_probabilities)
                state, reward, done, _ = env.step(action)
                if done:
                    reward = 0
#                 print(action,step_count)
    #                 env.render()
                step_count += 1
            print(step_count)
            print("---------------end-----------------")
            metrics['rewards_evaluation'].append(step_count)


In [187]:
ac2 = A2C(3000)
ac2.train()
ac2.execute(10)

/var/folders/w5/kksgfjf13x3g8sx7lr4h1ts00000gn/T/ipykernel_4854/1606409619.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))


99 [40, 60, 59, 50, 41, 60, 33, 97, 30, 35]
199 [121, 103, 150, 57, 75, 93, 75, 44, 70, 69]
299 [37, 30, 42, 30, 36, 34, 31, 31, 28, 28]
399 [38, 45, 32, 37, 40, 34, 29, 26, 30, 39]
499 [224, 239, 160, 160, 126, 355, 170, 187, 214, 191]
599 [173, 221, 235, 187, 228, 225, 212, 145, 169, 181]
699 [184, 178, 115, 117, 110, 93, 156, 112, 319, 111]
799 [234, 500, 500, 335, 500, 405, 406, 500, 262, 192]
899 [500, 500, 500, 458, 396, 378, 500, 500, 500, 500]
51
500
---------------end-----------------
500
---------------end-----------------
500
---------------end-----------------
500
---------------end-----------------
500
---------------end-----------------
500
---------------end-----------------
500
---------------end-----------------
500
---------------end-----------------
500
---------------end-----------------
500
---------------end-----------------


In [188]:
%matplotlib inline
plt.plot(metrics['total_rewards'])
plt.figtext(0.5, 0.01, "Total Rewards per episode", wrap=True, horizontalalignment='center', fontsize=12)
plt.savefig('total_rewards.png')
plt.show()

plt.plot(metrics['rewards_evaluation'])
plt.figtext(0.5, 0.01, "Rewards evaluation on test set", wrap=True, horizontalalignment='center', fontsize=12)
plt.savefig('rewards evaluation.png')


plt.show()


<Figure size 432x288 with 1 Axes>
<Figure size 432x288 with 1 Axes>
